### Gruppo 5.a: Castagnacci Giulia, Giordano Elisabetta

## Analisi e revisione della classe refactoring

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);  #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]   #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Funzioni aggiuntive create

### createIntervalTree
Aggiunta funzione createIntervalTree, struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [3]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [4]:
t = createIntervalTree(dict)    # Creazione albero

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

### removeIntersection
Elimina le intersezioni di ogni bounding box con loro stessi. Utilizzata in: spaceindex

In [5]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

### addIntersection
Aggiunge gli elementi di iterator nell’i-esimo array di covers. Utilizzata in: spaceindex, boxcovering

In [6]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)    #splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

## Funzione di supporto

In [7]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

### Funzione crossingTest

 Funzione di supporto per pointInPolygonClassification. Utilizzata per aggiornare il valore di “count” a seconda dello stato, new o old (entrambi di tipo Int), incrementando o decrementando il count di 0.5. 

In [8]:
function crossingTest(new::Int, old::Int, count::T, status::Int)::Number where T <: Real
    if status == 0
        status = new
        count += 0.5
    else
        if status == old
        	count += 0.5
        else
        	count -= 0.5
        end
        status = 0
    end
end

crossingTest (generic function with 1 method)

### Funzione setTile
Funzione utilizzata per impostare il tileCode di una bounding box 2D includendo il point 2D di coordinate x e y. A seconda della posizione di “point”, il tileCode varia da 0 a 15. Serve per impostare la tile del piano in base alla posizione del punto di query, per testare successivamente i codici del tile dei bordi di un poligono 2D e determinare se tale punto è interno o esterno o sul confine del poligono. Questa funzione si deve parallelizzare.

In [9]:
function setTile(box)
	tiles = [[9,1,5],[8,0,4],[10,2,6]]
	b1,b2,b3,b4 = box
	function tileCode(point)
		x,y = point
		code = 0
		if y>b1 code=code|1 end
		if y<b2 code=code|2 end
		if x>b3 code=code|4 end
		if x<b4 code=code|8 end
		return code
	end
	return tileCode
end

setTile (generic function with 1 method)

### Versione iniziale pointInPolygonClassification
Funzione utilizzata per la classificazione dei punti di un poligono. In particolare, identifica se un punto si trova all’interno, all’esterno o sul bordo di un poligono, richiama la setTile per dirlo.


In [10]:
function pointInPolygonClassification(V,EV)
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                if c_int == 0 return "p_on"
                elseif c_int == 4 count += 1 end
            elseif c_edge == 15
                x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
                if x_int > x count += 1
                elseif x_int == x return "p_on" end
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 count += 1
            elseif c_edge == 11 count = count
            elseif c_edge == 1
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(1,2,status,count) end
            elseif c_edge == 2
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(2,1,status,count) end
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(1,2,status,count) end
            elseif c_edge == 6
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(2,1,status,count) end
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification (generic function with 1 method)

In [11]:
@btime pointInPolygonClassification(V,EV)

  84.486 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#13"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

In [12]:
@code_warntype pointInPolygonClassification(V,EV)

MethodInstance for pointInPolygonClassification(

::Matrix{Float64}, ::Vector{Vector{Int64}})
  from pointInPolygonClassification(V, EV) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:

1
Arguments
  

#self#::Core.Const(pointInPolygonClassification)
  V::Matrix{Float64}
  EV::Vector{Vector{Int64}}
Locals
  pointInPolygonClassification0::var"#pointInPolygonClassification0#13"{Matrix{Float64}, Vector{Vector{Int64}}}
Body::var"#pointInPolygonClassification0#13"{Matrix{Float64}, Vector{Vector{Int64}}}


1 ─ %1 = 

Main.:(var"#pointInPolygonClassification0#13")

::Core.Const(var"#pointInPolygonClassification0#13")
│   %2 

= Core.typeof(V)::Core.Const(Matrix{Float64})
│   %3 = Core.typeof(EV)::Core.Const(Vector{Vector{Int64}})
│   %4 = Core.apply_type(%1, %2, %3)::Core.Const(var"#pointInPolygonClassification0#13"{Matrix{Float64}, Vector{Vector{Int64}}})
│        (pointInPolygonClassification0 = %new(%4, V, EV))
└──      

return pointInPolygonClassification0



In [13]:
@benchmark pointInPolygonClassification(V,EV)

BenchmarkTools.Trial: 10000 samples with 947 evaluations.
 Range (min … max):   85.639 ns …   4.536 μs  ┊ GC (min … max): 0.00% … 94.97%
 Time  (median):     123.442 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   167.984 ns ± 114.810 ns  ┊ GC (mean ± σ):  1.23% ±  2.33%

  ▃▆▆█▅▄▄▂▁▁▁           ▇▇▄▂▁▁                                  ▂
  █████████████████▆▇▆▅▆███████████▇▆▇▇▆▆▆▅▅▆▆▅▇▇▆▇▆▇▇▆▅▇▇▇▇▇▆▇ █
  85.6 ns       Histogram: log(frequency) by time        432 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

### Versione parallelizzata di pointInPolygonClassification

In [14]:
function edgecode1(c_int) #c_edge == 1
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(1,2,status, counter) end 
end 

edgecode1 (generic function with 1 method)

In [15]:
function edgecode2(c_int) #c_edge == 2
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(2,1,status, counter) end 
end 

edgecode2 (generic function with 1 method)

In [16]:
function edgecode3(c_int) #c_edge == 3
    if c_int == 0 return "p_on"
    elseif c_int == 4 counter += 1 end 
end 

edgecode3 (generic function with 1 method)

In [17]:
function edgecode4(c_un) #c_edge == 4
    if c_un == 4 return "p_on" end 
end 

edgecode4 (generic function with 1 method)

In [18]:
function edgecode5(c1,c2) #c_edge == 5
    if (c1==0) | (c2==0) return "p_on"
    else crossingTest(1,2,status, counter) end 
end 

edgecode5 (generic function with 1 method)

In [19]:
function edgecode6(c1,c2) #c_edge == 6
    if ((c1==0) | (c2==0)) return "p_on"
    else crossingTest(2,1,status, counter) end 
end 

edgecode6 (generic function with 1 method)

In [20]:
function edgecode7(counter) #c_edge == 7
    counter += 1
end 

edgecode7 (generic function with 1 method)

In [21]:
function edgecode8(c_un) #c_edge == 8
    if (c_un == 8) return "p_on" end   
end

edgecode8 (generic function with 1 method)

In [22]:
function edgecode9_10(c1,c2) #c_edge == 9/10
    if ((c1 ==0) | (c2==0)) return "p_on" end
end

edgecode9_10 (generic function with 1 method)

In [23]:
function edgecode11() #c_edge == 11
    count = count
end

edgecode11 (generic function with 1 method)

In [24]:
function edgecode12(c_un) #c_edge = 12
    if (c_un == 12 ) return "p_on" end    
end

edgecode12 (generic function with 1 method)

In [25]:
function edgecode13(c1,c2) #c_edge = 13
    if (( c1 ==4) | (c2 == 4))
        crossingTest(1,2,status, counter) end
end

edgecode13 (generic function with 1 method)

In [26]:
function edgecode14(c1,c2) #c_edge = 14
    if (( c1 ==4) | (c2 == 4))
        crossingTest(2,1,status, counter) end
end

edgecode14 (generic function with 1 method)

In [27]:
function edgecode15(x1,x2,y1,y2,x,y)
    x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
    if x_int > x counter+=1
    elseif (x_int == x) return "p_on" end
end

edgecode15 (generic function with 1 method)

In [28]:
function pointInPolygonClassification2(V,EV) 
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) edgecode1(c_un)
            elseif (c_edge == 12) edgecode12(c_un)
            elseif (c_edge == 3) edgecode3(c_int)
            elseif (c_edge == 15) edgecode15(x1,x2,y1,y2,x,y)
            elseif (c_edge == 13) edgecode13(c1,c2)
            elseif (c_edge == 14) edgecode14(c1,c2)
            elseif (c_edge == 7) edgecode7(counter)
            elseif (c_edge == 11) edgecode11()
            elseif (c_edge == 1) edgecode1(c_int)
            elseif (c_edge == 2) edgecode2(c_int)
            elseif (c_edge == 4) edgecode4(c_un)
            elseif (c_edge == 8) edgecode8(c_un)
            elseif (c_edge == 5) edgecode5(c1,c2)
            elseif (c_edge == 6) edgecode6(c1,c2)
            elseif ((c_edge == 9) | (c_edge == 10)) edgecode9_10(c1,c2)
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification2 (generic function with 1 method)

In [29]:
@btime pointInPolygonClassification2(V,EV)

  83.560 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#14"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

In [30]:
@benchmark pointInPolygonClassification2(V,EV)

BenchmarkTools.Trial: 10000 samples with 950 evaluations.
 Range (min … max):   83.368 ns …   9.633 μs  ┊ GC (min … max): 0.00% … 97.57%
 Time  (median):     119.789 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   161.126 ns ± 152.489 ns  ┊ GC (mean ± σ):  1.96% ±  2.37%

  ▂▅▅▆▇▆▃▃▂▁▁              ▄█▄▂▁▁▁▁                             ▂
  ████████████████▇▇▇▇▇▆▆▇▆█████████▇█▇█▆▅▇▆▆▅▆▆▅▅▄▅▅▄▆▅▅▅▆▆▅▄▄ █
  83.4 ns       Histogram: log(frequency) by time        378 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

### Funzione input_collection
Prende in input un array di tipo Lar.LAR e viene utilizzata per la selezione di una faccia e per la costruzione di una collezione di dimensione d-1, da una di dimensione d o d-1. Ritorna in output il risultato dell’applicazione della funzione Lar.struct2lar(), la quale valuta un oggetto geometrico di tipo Lar.Struct, generando la tripla (V, FV, EV). Il risultato dato può essere dato in input ad algoritmi di pipeline 2D-3D.

In [31]:
function input_collection(data::Array)::Lar.LAR
	assembly = Lar.Struct(data)
	return Lar.struct2lar(assembly)
end

input_collection (generic function with 1 method)

### Versione iniziale boundingbox
Prende in input un vertice di tipo Lar.Points e restituisce in output due array che indicano gli estremi del bounding box. Serve, quindi, a creare il bounding box di un segmento, cioè la scatola entro cui sono contenuti tutti i punti.


In [32]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

In [33]:
@btime boundingbox(V)

  11.500 μs (147 allocations: 5.03 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

In [34]:
@code_warntype boundingbox(V)

MethodInstance for boundingbox(::Matrix{Float64})
  from boundingbox(vertices::Matrix) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(boundingbox)
  vertices::Matrix{Float64}
Locals
  #16::var"#16#18"
  #15::var"#15#17"
  maximum::Any
  minimum::Any
Body::Tuple{Any, Any}
1 ─       (#15 = %new(Main.:(var"#15#17")))
│   %2  = #15::Core.Const(var"#15#17"())
│   %3  = 

(:dims,)::Core.Const((:dims,))
│   %4  = Core.apply_type(Core.NamedTuple, %3)::Core.Const(NamedTuple{(:dims,)})
│   %5

  = Core.tuple(2)::Core.Const((2,))
│   %6  = (%4)(%5)

::Core.Const((dims = 2,))
│   %7  = Core.kwfunc(Main.mapslices)::Core.Const(Base.var"#mapslices##kw"())
│         (minimum = (%7)(%6, Main.mapslices, %2, vertices))
│         (#16 = %new(Main.:(var"#16#18")))
│   %10 = #16::Core.Const(var"#16#18"())
│   %11 = (:dims,)::Core.Const((:dims,))
│   %12 = Core.apply_type(Core.NamedTuple, %11)::Core.Const(NamedTuple{(:dims,)})
│   %13 = Core.tuple(2)::Core.Const((2,))
│   %14 = (%12)(%13)::Core.Const((dims = 2,))
│   %15 = Core.kwfunc(Main.mapslices)::Core.Const(Base.var"#mapslices##kw"())
│         (maximum = (%15)(%14, Main.mapslices, %10, vertices))
│   %17 = Core.tuple(minimum, maximum)::Tuple{Any, Any}
└──       return 

%17



In [35]:
@benchmark boundingbox(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.900 μs …  9.101 ms  ┊ GC (min … max): 0.00% … 99.32%
 Time  (median):     14.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.389 μs ± 97.261 μs  ┊ GC (mean ± σ):  4.92% ±  0.99%

  ▅▆█▇▇▆▅▄▃▃▂▂ ▁      ▁▁▂▁▁▁▁  ▁                              ▂
  ██████████████▇█▇▇███████████████▇▇▆▆▆▅▆▆▆▄▅▆▆▆▅▅▅▅▅▅▁▅▆▃▃▆ █
  11.9 μs      Histogram: log(frequency) by time      55.8 μs <

 Memory estimate: 5.03 KiB, allocs estimate: 147.

### Versione parallelizzata di boundingbox

In [36]:
function boundingbox2(vertices::Lar.Points)
     firstDim = vertices[1,:]
     secondDim = vertices[2,:]
     if (size(vertices,1)==3)
         thirdDim = vertices[3,:]
          minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
          maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
     else
          minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
          maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
     end
     return fetch(minimum),fetch(maximum)
  end

boundingbox2 (generic function with 1 method)

In [37]:
@btime boundingbox2(V)

  4.286 μs (63 allocations: 2.50 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

In [38]:
@benchmark boundingbox(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.400 μs …  9.474 ms  ┊ GC (min … max): 0.00% … 99.40%
 Time  (median):     14.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.514 μs ± 95.336 μs  ┊ GC (mean ± σ):  5.09% ±  0.99%

  ▄██▇▆▅▄▃▂▂▃▂▁ ▁▁▁▂▁▁▁                                       ▂
  █████████████████████████████▇▇▇▆▇▇▇▇▅▇▇▆▇▇▅▆▅▅▅▄▆▅▆▆▅▄▅▄▅▅ █
  11.4 μs      Histogram: log(frequency) by time      60.9 μs <

 Memory estimate: 5.03 KiB, allocs estimate: 147.

### Funzione coordintervals
Prende in input una matrice (i bounding box) e un intero che specifica la coordinata su cui si lavora. La funzione restituisce un dizionario ordinato, dove la chiave è l'intervallo sulla singola coordinata, mentre il valore associato è un array di indici che indicano a quali bounding box si riferiscono.

In [39]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

In [40]:
@btime coordintervals(1,bb)

  736.082 ns (17 allocations: 1.28 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 4 entries:
  [0.0, 1.0] => [1, 3]
  [1.0, 1.0] => [2]
  [0.0, 0.0] => [4]
  [0.0, 2.0] => [5]

In [41]:
@benchmark coordintervals(1,bb)

BenchmarkTools.Trial: 10000 samples with 85 evaluations.
 Range (min … max):  782.353 ns … 164.638 μs  ┊ GC (min … max):  0.00% … 99.00%
 Time  (median):       1.028 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.588 μs ±   4.950 μs  ┊ GC (mean ± σ):  13.58% ±  4.51%

  ▄▆▇█▇▆▅▄▃▃▃▂▂▁▁▁▁▁  ▄▆▄▃▃▃▂▁▁ ▂▂▂▁▁                           ▂
  ████████████████████████████████████▇█▇██▇▇▆▇▇▇▆▆▇▇█▇▇▆▆▅▅▅▇▆ █
  782 ns        Histogram: log(frequency) by time       3.87 μs <

 Memory estimate: 1.28 KiB, allocs estimate: 17.

In [42]:
@code_warntype coordintervals(1,bb)

MethodInstance for coordintervals(::Int64, ::Vector{Matrix{Float64}})
  from coordintervals(coord, bboxes) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(coordintervals)
  coord::Int64
  bboxes::Vector{Matrix{Float64}}
Locals
  @_4::Union{Nothing, Tuple{Tuple{Int64, Matrix{Float64}}, Tuple{Int64, Int64}}}
  boxdict::OrderedDict{Vector{Float64}, Vector{Int64}}
  @_6::Int64
  box::Matrix{Float64}
  h::Int64
  key::Vector{Float64}
Body::OrderedDict{Vector{Float64}, Vector{Int64}}
1 ─ %1  = Core.apply_type(Main.Array, Main.Float64, 1)::Core.Const(Vector{Float64})


│   %2  = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Const(Vector{Int64})
│   %3  = Core.apply_type(Main.OrderedDict, %1, %2)::Core.Const(OrderedDict{Vector{Float64}, Vector{Int64}})
│         (boxdict = (%3)())
│   %5  = Main.enumerate(bboxes)::Base.Iterators.Enumerate{Vector{Matrix{Float64}}}
│         (@_4 = Base.iterate(%5))
│   %7  = (@_4

 === nothing)::Bool
│   %8  = Base.not_int(%7)::Bool
└──       goto #7 if not %8
2 ┄ %10 = @_4::Tuple{Tuple{Int64, Matrix{Float64}}, Tuple{Int64, Int64}}
│   %11 = Core.getfield(%10, 1)::Tuple{Int64, Matrix{Float64}}
│   %12 = Base.indexed_iterate(%11, 1

)::Core.PartialStruct(Tuple{Int64, Int64}, Any[Int64, Core.Const(2)])
│         (h = Core.getfield(%12, 1))
│         (@_6 = Core.getfield(%12, 2))
│   %15 = Base.indexed_iterate(%11, 2, 

@_6::

Core.Const(2))::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(3)])
│         (box = Core.getfield(%15, 1))
│   %17 = Core.getfield(%10, 2)::Tuple{Int64, Int64}
│         (key = Base.getindex(box, coord, Main.:(:)))
│   %19 = Main.haskey(boxdict, key)::Bool
│   %20 = (%19 == 

false)::Bool
└──       goto #4 if not %20
3 ─ %22 = Base.vect(h)::Vector{Int64}
│         Base.setindex!(boxdict, %22, key)
└──       goto #5
4 ─ %25 = Base.getindex(boxdict, key)::Vector{Int64}
└──       Main.push!(%25, h)
5 ┄       (@_4 = Base.iterate(%5, %17))
│   %28 = (@_4 === nothing)::Bool
│   %29 = Base.not_int(%28)::Bool
└──       goto #7 if not %29
6 ─       goto #2
7 ┄       return boxdict



### Versione iniziale di boxcovering
 Prende in input una matrice (i bounding box), un intero che specifica la coordinata su cui si lavora, e un intervalTrees. La funzione restituisce una matrice che contiene tutte le intersezioni tra bounding box.

In [43]:
function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

boxcovering (generic function with 1 method)

In [44]:
cov = boxcovering(bb, 1, t)

5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [45]:
ex = bb[1][1,:]

2-element Vector{Float64}:
 0.0
 1.0

In [46]:
it = IntervalTrees.intersect(t, tuple(ex...))

IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}(IntervalTrees.true_cmp, IntervalTrees.Intersection{Float64, IntervalValue{Float64, Array}, 64}(0, #undef), IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}, Interval{Float64}
(0.0,1.0))


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]


(1.0,1.0) => [2]

In [47]:
@btime boxcovering(bb, 1, t) 

  3.250 μs (82 allocations: 3.02 KiB)


5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [48]:
@code_warntype boxcovering(bb, 1, t) 

MethodInstance for boxcovering(::Vector{Matrix{Float64}}, ::Int64, ::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64})
  from boxcovering(bboxes, index, tree) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(boxcovering)
  bboxes::Vector{Matrix{Float64}}
  index::Int64
  tree::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
Locals
  @_5::Union{Nothing, Tuple{Tuple{Int64, Matrix{Float64}}, Tuple{Int64, Int64}}}
  #27::var"#27#28"
  covers::Vector{Vector{Any}}
  @_8::Union{Nothing, Tuple{IntervalValue{Float64, Array}, Int64}, Tuple{IntervalValue{Float64, Array}, Nothing}}
  @_9::Int64
  boundingbox::Matrix{Float64}
  i::Int64
  iterator::Union{IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}, Vector{IntervalValue{Float64, Array}}}
  extent::Vector{Float64}
  x::IntervalValue{Float64, Array}
Body::Vecto

│   %3  = Main.length(bboxes)::Int64
│   %4  = (1:%3)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %5  = Base.Generator(%2, %4)::Core.PartialStruct(Base.Generator{UnitRange{Int64}, var"#27#28"}, Any[Core.Const(var"#27#28"()), Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])])
│         (covers = Base.collect(%5))
│   %7  = Main.enumerate(bboxes)::Base.Iterators.Enumerate{Vector{Matrix{Float64}}}
│         (@_5 = Base.iterate(%7))
│   %9  = (@_5 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #7 if not %10
2 ┄

 %12 = @_5::Tuple{Tuple{Int64, Matrix{Float64}}, Tuple{Int64, Int64}}
│   %13 = Core.getfield(%12, 1)::Tuple{Int64, Matrix{Float64}}
│   %14 = Base.indexed_iterate(%13, 1)::Core.PartialStruct(Tuple{Int64, Int64}, Any[Int64, Core.Const(2)])
│         (i = Core.getfield(%14, 1))
│         (@_9 = Core.getfield(%14, 2))
│   %17 = Base.indexed_iterate(%13, 2, @_9::Core.Const(2))::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(3)])
│         (boundingbox = Core.getfield(%17, 1))
│   %19 = Core.getfield(%12, 2)::Tuple{Int64, Int64}
│   %20 = Base.getindex(bboxes, i)::Matrix{Float64}
│         (extent = Base.getindex(%20, index, Main.:(:)))
│  

 %22 = IntervalTrees.intersect::Core.Const(intersect)
│   %23 = Core._apply_iterate(Base.iterate, Main.tuple, extent)::Tuple{Vararg{Float64}}


│         (iterator = (%22)(tree, %23))
│   %25 = iterator::Union{IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}, Vector{IntervalValue{Float64, Array}}}
│         (@_8 = Base.iterate(%25))
│   %27 = (@_8 === nothing)::Bool
│   %28 = Base.not_int(%27)::Bool
└──       goto #5 if not %28
3 ┄ %30 = @_8::Union{Tuple{IntervalValue{Float64, Array}, Int64}, Tuple{IntervalValue{Float64, Array}, Nothing}}
│         (x = Core.getfield(%30, 1))
│   %32 = Core.getfield(%30, 2)::Union{Nothing, Int64}
│   %33 = Base.getindex(covers, i)::Vector{Any}
│   %34 = Base.getproperty(

x, :value)::Array
│         Main.append!(%33, %34)
│         (@_8 = Base.iterate(%25, %32))
│  

 %37 = (@_8 === nothing)::Bool
│   %38 = Base.not_int(%37)::Bool
└──       goto #5 if not %38
4 ─       goto #3
5 ┄       (@_5 = Base.iterate(%7, %19))
│   %42 = (@_5 === nothing)::Bool
│   %43 = Base.not_int(%42)::Bool
└──       goto #7 if not %43
6 ─       goto #2
7 ┄       return covers



In [49]:
@benchmark boxcovering(bb, 1, t) 

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.312 μs …  1.795 ms  ┊ GC (min … max): 0.00% … 99.57%
 Time  (median):     4.312 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.563 μs ± 25.977 μs  ┊ GC (mean ± σ):  8.98% ±  1.99%

      ▄█▃▇▂                                                   
  ▁▃▃▆█████▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.31 μs        Histogram: frequency by time          11 μs <

 Memory estimate: 3.02 KiB, allocs estimate: 82.

### Versione parallelizzata di boxcovering

In [50]:
function boxcovering2(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcovering2 (generic function with 1 method)

In [51]:
@btime boxcovering2(bb, 1, t)

  4.683 μs (86 allocations: 4.83 KiB)


5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [52]:
@benchmark boxcovering2(bb, 1, t)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   5.129 μs …  7.092 ms  ┊ GC (min … max):  0.00% … 99.80%
 Time  (median):     12.000 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   14.261 μs ± 83.178 μs  ┊ GC (mean ± σ):  11.59% ±  2.42%

  ▃▄▃▁           ██▆▅▅▄▃▂▁▁               ▁▁                  ▂
  ████▇▇▇█▆▄▇▆▆▇▆█████████████▇▇▇▇▆▇▆▇▇▇█████▇▆▆▅▆▅▅▅▄▅▄▄▅▄▅▅ █
  5.13 μs      Histogram: log(frequency) by time      30.1 μs <

 Memory estimate: 4.83 KiB, allocs estimate: 86.

### Versione iniziale di spaceindex
Prende in input una tupla costituita da una matrice che contiene i punti del modello, di tipo Lar.LAR, e da un array di array che contiene le celle cioè scomposizioni dello spazio geometrico. La funzione restituisce in output un array di array dove l'elemento i-esimo rappresenta quali intersezioni ha il bounding box i-esimo con gli altri bounding box. Nello specifico, la funzione calcola le 1-celle e il loro bounding box attraverso la funzione boundingBox. Si suddividono le coordinate x e y in due dizionari chiamando la funzione coordintervals. Per entrambe le coordinate x e y, si calcola un intervalTree cioè una struttura dati che contiene intervalli. La funzione boxCovering viene chiamata per calcolare le sovrapposizioni sulle singole dimensioni dei bounding Box. Intersecando quest’ultime, si ottengono le intersezioni effettive tra bounding box. La funzione esegue lo stesso procedimento sulla coordinata z se presente. Infine, si eliminano le intersezioni di ogni bounding box con loro stessi.


In [53]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end

spaceindex (generic function with 1 method)

In [54]:
@btime spaceindex((V,EV))

  82.000 μs (1097 allocations: 49.00 KiB)

5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

In [55]:
@code_warntype spaceindex((V,EV))

MethodInstance for spaceindex(::Tuple{Matrix{Float64}, Vector{Vector{Int64}}})
  from spaceindex(model::Union{Tuple{Matrix, Vector{Vector{Int64}}}, Tuple{Matrix, Vector{Vector{Int64}}, Vector{Vector{Int64}}}}) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(spaceindex)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  @_3::Any
  #35::var"#35#39"
  @_5::Any
  #34::var"#34#38"
  @_7::Any
  @_8

::Any
  #33::var"#33#37"
  #32::var"#32#36"{Vector{Vector{Int64}}, Matrix{Float64}}
  @_11::Int64
  covers::Any
  zcovers::Any
  zs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  zboxdict::Any
  ycovers::Any
  xcovers::Any
  ys::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  xs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  yboxdict::Any
  xboxdict::Any
  bboxes::Vector
  cellpoints::Vector{Matrix{Float64}}
  dim::Int64
  CV::Vector{Vector{Int64}}
  V::Matrix{Float64}
  @_27::Any
  boxset@_28::Any
  key@_29::Any
  @_30::Any
  boxset@_31::Any


  key@_32::Any
  @_33::Any
  boxset@_34::Any
  key@_35::Any
  k::Any
Body::Vector{Vector{Int64}}
1 ──        

Core.NewvarNode(:(@_3))
│           Core.NewvarNode(:(#35))
│           Core.NewvarNode(:(@_5))
│           Core.NewvarNode(:(#34))
│           Core.NewvarNode(:(@_7))
│           Core.NewvarNode(:(covers))
│   

        Core.NewvarNode(:(zcovers))
│           Core.NewvarNode(:(zs))
│           Core.NewvarNode(:(zboxdict))
│           Core.NewvarNode(:(ycovers))
│           Core.NewvarNode(:(xcovers))
│           Core.NewvarNode(:(ys))
│    %13  = Core.apply_type(Main.Array, Main.Int, 1)::Core.Const(Vector{Int64})
│    %14  = Core.apply_type(Main.Array, %13, 1)::Core.Const(Vector{Vector{Int64}})
│    %15  = (1:2

...

goto #12 if not %103
8 ── %105 = Base.getproperty(Main.Lar, :coordintervals)::Any
│           (zboxdict = (%105)(3, bboxes))
│    %107 = IntervalTrees.IntervalMap::Core.Const(IntervalMap)
│    %108 = Core.apply_type(%107, Main.Float64, Main.Array)::Core.Const(IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64})
│           (zs = (%108)())
│    %110 = zboxdict::Any
│           (@_5 = Base.iterate(%110))
│    %112 = (@_5 === nothing)::Bool
│    %113 = Base.not_int(%112)

::Bool
└───        goto #11 if not %113
9 ┄─ %115 = @_5::Any
│    %116 = Core.getfield(%115, 1)::Any
│    %117 = Base.indexed_iterate(%116, 1)::Any
│           (key@_35 = Core.getfield(%117, 1))
│           (@_33 = Core.getfield(%117, 2))
│    %120 = Base.indexed_iterate(%116, 2, @_33)::Any
│           (boxset@_34 = Core.getfield(%120, 1))


│    %122 = Core.getfield(%115, 2)::Any
│    %123 = zs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
│    %124 = boxset@_34::Any
│    %125 = Core._apply_iterate(Base.iterate, Main.tuple, key@_35)::Tuple
│           Base.setindex!(%123, %124, %125)
│           (@_5 = Base.iterate(%110, %122))
│    %128 = (@_5 === nothing)::Bool
│    %129 = Base.not_int(%128)::Bool
└───        goto #11 if not %129
10 ─        goto #9
11 ┄ %132 = Base.getproperty(Main.Lar, :boxcovering)::Any
│    %133 = bboxes::Vector
│           (zcovers = (%132)(%133, 3, zs))
│           (#35 = %new(Main.:

(var"#35#39")))
│    %136 = #35::Core.Const(var"#35#39"())
│    %137 = Main.zip(zcovers, covers)::Union{DataStructures.ZippedSparseIntSetIterator{Tuple{DataStructures.SparseIntSet, DataStructures.SparseIntSet}, Tuple{}}, Base.Iterators.Zip}
│    %138 = Base.Generator(%136, %137)::Base.Generator{_A, var"#35#39"} where _A
└───        (covers = Base.collect(%138))
12 ┄ %140 = Main.length(covers)::Any
│    %141 = (1:%140)::Any
│           (@_3 = Base.iterate(%141))
│    %143 = (@_3 === nothing)::Bool
│    %144 = Base.not_int(%143)::Bool
└───        goto #15 if not %144
13 ┄ %146 = @_3::Any
│           (k = Core.getfield(%146, 1))
│    %148 = Core.getfield(%

146, 2)::Any
│    %149 = Base.getindex(covers, k)::Any
│    %150 = Base.vect(k)::Vector
│    %151 = Main.setdiff(%149, %150)::Any
│           Base.setindex!(covers, %151, k)
│           (@_3 = Base.iterate(%141, %148))
│    %154 = (@_3 === nothing)::Bool
│    %155 = Base.not_int(%154)::Bool
└───        goto #15 if not %155
14 ─        goto #13
15 ┄ %158 = Base.convert(%14, covers)::Any
│    %159 = Core.typeassert(%158, %14)::Vector{Vector{Int64}}
└───        return %159



In [56]:
@benchmark spaceindex((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   79.500 μs …  20.867 ms  ┊ GC (min … max): 0.00% … 94.20%
 Time  (median):      94.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.688 μs ± 381.138 μs  ┊ GC (mean ± σ):  8.04% ±  2.93%

  ▄██▆▅▄▄▃▂▂▂ ▁         ▂▃▃▂▂▁                                  ▂
  █████████████████████████████▇█▇▇▇▆▇▇▆▅▆▆▅▆▅▅▅▅▄▅▄▅▄▄▅▅▅▅▄▅▄▅ █
  79.5 μs       Histogram: log(frequency) by time        419 μs <

 Memory estimate: 49.00 KiB, allocs estimate: 1097.

### Versione parallelizzata di spaceindex

In [57]:
function spaceindex2(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
    dim = size(V,1)
    
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]		    # calcola le celle
    bboxes = [hcat(boundingbox2(cell)...) for cell in cellpoints]       # calcola i boundingbox delle celle
    
    xboxdict = Threads.@spawn coordintervals(1,bboxes)
    yboxdict = Threads.@spawn coordintervals(2,bboxes)

    # xs,ys sono di tipo IntervalTree
    xs = Threads.@spawn createIntervalTree(fetch(xboxdict))
    ys = Threads.@spawn createIntervalTree(fetch(yboxdict))
    
    xcovers = Threads.@spawn boxcovering2(bboxes, 1, fetch(xs))                        # lista delle intersezioni dei bb sulla coordinata x
    ycovers = Threads.@spawn boxcovering2(bboxes, 2, fetch(ys))                        # lista delle intersezioni dei bb sulla coordinata x
    covers = [intersect(pair...) for pair in zip(fetch(xcovers),fetch(ycovers))]       # lista delle intersezioni dei bb su entrambe le coordinate

    if dim == 3
        zboxdict = Threads.@spawn coordintervals(3,bboxes)
        zs = Threads.@spawn createIntervalTree(fetch(zboxdict))
        zcovers = Threads.@spawn boxcovering2(bboxes, 3, fetch(zs))
        covers = [intersect(pair...) for pair in zip(fetch(zcovers),covers)]
    end
    
    removeIntersection(covers)  # rimozione delle intersezioni con se stesso
    return covers
end

spaceindex2 (generic function with 1 method)

In [58]:
@btime spaceindex2((V,EV))

  47.800 μs (538 allocations: 40.08 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

In [59]:
@benchmark spaceindex2((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  47.400 μs …  16.780 ms  ┊ GC (min … max): 0.00% … 97.71%
 Time  (median):     59.400 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   85.888 μs ± 268.932 μs  ┊ GC (mean ± σ):  7.73% ±  2.60%

  ▃██▇▆▅▅▄▃▂▂▂▁▁▂▂▁▁▁ ▁▁▂▃▄▄▂▂▁                                ▂
  ██████████████████████████████▇▇▇▇▆▆▆▆▆▅▅▄▅▆▆▅▅▆▆▇▆▆▅▅▅▅▄▅▅▅ █
  47.4 μs       Histogram: log(frequency) by time       263 μs <

 Memory estimate: 40.08 KiB, allocs estimate: 538.

### Funzione intersection
Funzione utilizzata per l’intersezione di due segmenti di linea in 2D, Restituisce in output i due parametri di linea del punto di intersezione. I segmenti si intersecano se entrambi i parametri di ritorno, α e β, sono contenuti nell'intervallo [0, 1].


In [60]:
function intersection(line1,line2)
	x1,y1,x2,y2 = vcat(line1...)
	x3,y3,x4,y4 = vcat(line2...)

	# intersect lines e1,e2
	det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
	if det != 0.0
		a = 1/det
		b = [y1-y2 x2-x1; y3-y4 x4-x3]
		c = [x1-x3; y1-y3]
		(β,α) = a * b * c
	else
		if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
			 return nothing
		else
			 # segments parallel: no intersection
			 return nothing
		end
	end
	return α,β
end

intersection (generic function with 1 method)

### Versione iniziale congruence
Prende in input un model di tipo Lar.LAR, inizializza un BallTree, che divide ricorsivamente i punti in gruppi delimitati da ipersfere, un raggio di ricerca e un array vuoto di W elementi (W = matrice 2xW). Per ogni vertice cerca i vertici più vicini nel raggio R e li sostituisce. Crea un dizionario chiave, valore (id nuovi vertici) che verrà poi utilizzato per etichettare i vertici degli spigoli in EW.

In [61]:
function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
	for k=1:size(W,2)
		near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near) 
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
	for e in (EW)
		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)
	
	return hcat(V...),EV
end

congruence (generic function with 1 method)

In [62]:
@btime congruence((V,EV))


  16.600 μs (176 allocations: 9.83 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

In [63]:
@code_warntype congruence((V,EV))

MethodInstance for congruence(::Tuple{Matrix{Float64}, Vector{Vector{Int64}}})
  from congruence(model) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(congruence)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  #69::var"#69#73"
  #68::var"#68#72"
  @_5::Union{Nothing, Tuple{Vector{Int64}, Int64}}
  #67::var"#67#71"{Matrix{Float64}}
  #66::var"#66#70"
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  @_10::Int64
  EV@_11::Core.Box


  V::Vector{Vector{Float64}}
  invidx::OrderedDict
  pointidx::Vector
  near@_15::Core.Box
  r::Float64
  balltree::BallTree{_A, _B, _C, Euclidean} where {_A<:(AbstractVector), _B, _C}
  EW::Vector{Vector{Int64}}
  W::Matrix{Float64}
  k@_20::Int64
  k@_21::Int64
  e::Vector{Int64}
  newedge::Vector
  near@_24::Union{}
  near@_25::Union{}
  near@_26::Union{}
  EV@_27::Union{}
  EV@_28::Union{}
  EV@_29::Union{}
  EV@_30::Union{}
Body::Tuple{Union{Matrix, Vector{Any}}, Any}
1 ──        Core.NewvarNode(:(#69))
│           Core.NewvarNode(:(#68))
│           Core.NewvarNode(:(@_5))
│           Core.NewvarNode

(:(#67))
│           Core.NewvarNode(:(#66))
│           Core.NewvarNode(:(@_8))
│           (EV@_11 = Core.Box())
│           Core.NewvarNode(:(V))
│           Core.NewvarNode(:(invidx))
│           Core.NewvarNode(:(pointidx))
│           (near@_15 = Core.Box())
│    %12  = Base.indexed_iterate(model, 1)::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(2)])
│           (W = Core.getfield(%12, 1))
│   

        (@_10 = Core.getfield(%12, 2))
│    %15  = Base.indexed_iterate(model, 2, @_10::Core.Const(2))::Core.PartialStruct(Tuple{Vector{Vector{Int64}}, Int64}, Any[Vector{Vector{Int64}}, Core.Const(3)])
│           (EW = Core.getfield(%15, 1))
│    %17  = NearestNeighbors.BallTree::Core.Const(BallTree)
│           (balltree = (%17)(W))
│           (r = 1.0e-10)
│    %20  = Core.apply_type(Main.Array, Main.Any)::Core.Const(Array{Any})
│    %21  = Main.size(W, 2)::Int64
│    %22  = (%20)(Main.undef, %21)::Vector{Any}
│           Core.setfield!(near@_15, :contents, %22)
│    %24  

= Main.size(W, 2)::Int64
│    %25  = (1:%24)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│           (@_9 = Base.iterate(%25))
│    %27  = (@_9 === nothing)::Bool
│    %28  = Base.not_int(%27)::Bool
└───        goto #7 if not %28
2 ┄─ %30  = @_9::Tuple{Int64, Int64}
│           (k@_20 = Core.getfield(%30, 1))
│    %32  = Core.getfield(%30, 2)::Int64
│    %33  = NearestNeighbors.inrange::Core.Const(NearestNeighbors.inrange)
│    %34  = balltree::BallTree{_A, _B, _C, Euclidean} where {_A<:(AbstractVector), _B, _C}
│    %35  = Base.getindex(W, Main.:(:), k@_20)::Vector{Float64}
│    %36  = r::Core.Const(1.0e-10)
│    %37  = (%33)(%34, %35, %36, true)::Vector{Int64}
│   

...

getindex(%117, %118)::Any
│           (newedge = Base.vect(%116, %119))
│    %121 = Base.getindex(newedge, 1)::Any
│    %122 = Base.getindex(newedge, 2)::Any
│    %123 = (%121 !== %122)::Bool
└───        goto #22 if not %123
18 ─ %125 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #20 if not %125
19 ─        goto #21
20 ─        Core.NewvarNode(:(EV@_27))
└───        EV@_27
21 ┄ %130 = Core.getfield(EV@_11, :contents)::Any
└───        Main.push!(%130, newedge

)
22 ┄        (@_5 = Base.iterate(%106, %113))
│    %133 = (@_5 === nothing)::Bool
│    %134 = Base.not_int(%133)::Bool
└───        goto #24 if not %134
23 ─        goto #17
24 ┄        (#68 = %new(Main.:(var"#68#72"), EV@_11))
│    %138 = #68::var"#68#72"
│           (#69 = %new(Main.:(var"#69#73"), EV@_11))
│    %140 = #69::var"#69#73"
│    %141 = Core.isdefined(EV@_11

, :contents)::Bool
└───        goto #26 if not %141
25 ─        goto #27
26 ─        Core.NewvarNode(:(EV@_28))
└───        EV@_28
27 ┄ %146 = Core.getfield(EV@_11, :contents)::Any
│    %147 = Main.length(%146)::Any
│    %148 = (1:%147)::Any
│    %149 = Base.Filter(%140, %148)::Base.Iterators.Filter{var"#69#73"}
│    %150 = Base.Generator(%138, %149)::Base.Generator{_A, var"#68#72"} where _A
│    %151 = Base.collect(%150)::Any
│           Core.setfield!(EV@_11, :contents, %151)
│    %153 = Base.getproperty(Main.Lar, :Cells)::Any
│    %154 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #29 if not %154
28 ─        goto #30
29 ─        Core.NewvarNode(:(EV@_29))
└───        EV@_29
30 ┄

 %159 = Core.getfield(EV@_11, :contents)::Any
│    %160 = Main.convert(%153, %159)::Any
│           Core.setfield!(EV@_11, :contents, %160)
│    %162 = Core._apply_iterate(Base.iterate, Main.hcat, V)::Union{Matrix, Vector{Any}}
│    %163 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #32 if not %163
31 ─        goto #33
32 ─        Core.NewvarNode(:(EV@_30))
└───        EV@_30
33 ┄ %168 = Core.getfield(EV@_11, :contents)::Any
│    %169

 = Core.tuple(%162, %168)::Tuple{Union{Matrix, Vector{Any}}, Any}
└───        return %169



In [64]:
@benchmark congruence((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  38.100 μs …  16.877 ms  ┊ GC (min … max): 0.00% … 99.26%
 Time  (median):     41.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   48.525 μs ± 219.126 μs  ┊ GC (mean ± σ):  6.30% ±  1.40%

  ▆█▆▅▆▄▄▄▃▃▂▂▁▁                                               ▂
  ████████████████▇▇▆▇▇▇██▆▇▇▇█▇▇▇▇▆▇▇▇▆▇▇▅▅▅▆▄▄▅▅▄▄▂▄▄▄▂▄▄▄▅▄ █
  38.1 μs       Histogram: log(frequency) by time       108 μs <

 Memory estimate: 9.83 KiB, allocs estimate: 176.

### Versione parallelizzata congruence

In [65]:
function congruence2(model)
    W,EW = model
    n = size(W,2)
    balltree = NearestNeighbors.BallTree(W)
    r = 0.0000000001
    near = Array{Any}(undef, n)
    @inbounds @threads for k=1:n
        near[k] = NearestNeighbors.inrange(balltree, W[:,k], r, true)
    end
    near = map(sort,near) 
    @inbounds @threads for k=1:n
        W[:,k] = W[:,near[k][1]]
    end
    pointidx = Array{Int64}(undef, n)
    @inbounds @threads for k=1:n
         pointidx[k] = near[k][1] 
    end
    l = length(pointidx)
    invidx = OrderedDict(zip(1:l, pointidx))
    V = Array{Array{Float64,1}}(undef, l)
    @inbounds @threads for k=1:l
        V[k] = W[:,k] 
    end
    
    EV = []
    m = length(EW)
    @inbounds for i = 1:m
        newedge = [invidx[EW[i][1]],invidx[EW[i][2]]]
        if newedge[1] !== newedge[2]
            push!(EV,newedge)
        end
    end
    filter!(x ->  length(x)==2, EV)
    EV = convert(Lar.Cells, EV)
    return hcat(V...),EV
end

congruence2 (generic function with 1 method)

In [66]:
@btime congruence2((V,EV))

  18.000 μs (152 allocations: 9.27 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

In [67]:
@benchmark congruence2((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  18.200 μs …  13.995 ms  ┊ GC (min … max): 0.00% … 99.31%
 Time  (median):     44.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.593 μs ± 141.253 μs  ┊ GC (mean ± σ):  2.92% ±  0.99%

  ▅▃▅▁▃ ▂▁     ▆█▇▆▅▅▄▂▂▁▁▁      ▁                             ▂
  ████████▆▇▇▆▆██████████████████████████▇▆▇▆▆▆▆▅▆▆▆▅▄▄▅▄▃▄▅▄▅ █
  18.2 μs       Histogram: log(frequency) by time       123 μs <

 Memory estimate: 9.27 KiB, allocs estimate: 152.

### Versione iniziale linefragments
Funzione che ordina i segmenti intersecati di ogni segmento. Per ogni segmento, se interseca con qualcosa, prende i punti (x1, y1), (x2, y2) del segmento i-eisimo, lo confronta con tutti gli altri segmenti presenti nel suo indice spaziale “Sigma” che fornisce il sottoinsieme di linee i cui box di contenimento intersecano il box di ciascuna linea di input (dato da EV), e restituisce i parametri (alfa e beta) necessari a determinare il punto di intersezione tra coppie di segmenti. Se le rette si intersecano, si verifica che i parametri alfa e beta siano ammissibili, se lo sono vengono immagazzinati nella struttura dati “params”.


In [68]:
function linefragments(V,EV,Sigma)
	# remove the double intersections by ordering Sigma
	m = length(Sigma)
	sigma = map(sort,Sigma)
	reducedsigma = sigma 
	# pairwise parametric intersection
	params = Array{Float64,1}[[] for i=1:m]
	for h=1:m
		if sigma[h] ≠ []
			line1 = V[:,EV[h]]
			for k in sigma[h]
				line2 = V[:,EV[k]]
				out = Lar.intersection(line1,line2) 
				if out ≠ nothing
					α,β = out
					if 0<=α<=1 && 0<=β<=1
						push!(params[h], α)
						push!(params[k], β)
					end
				end
			end
		end
	end
	# finalize parameters of fragmented lines
	fragparams = []
	for line in params
		push!(line, 0.0, 1.0)
		line = sort(collect(Set(line)))
		push!(fragparams, line)
	end
	return fragparams
end

linefragments (generic function with 1 method)

In [69]:
Sigma = spaceindex((V,EV))
@btime linefragments(V,EV,Sigma)

  25.000 μs (800 allocations: 25.34 KiB)


5-element Vector{Any}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

In [70]:
@code_warntype linefragments(V,EV,Sigma)

MethodInstance for linefragments(::Matrix{Float64}, ::Vector{Vector{Int64}}, ::Vector{Vector{Int64}})
  from linefragments(V, EV, Sigma) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(linefragments)
  V::Matrix{Float64}
  EV::Vector{Vector{Int64}}
  Sigma::Vector{Vector{Int64}}
Locals
  @_5::Union{Nothing, Tuple{Vector{Float64}, Int64}}
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  fragparams::Vector{Any}
  params::Vector{Vector{Float64}}
  reducedsigma::Vector{Vector{Int64}}
  sigma::Vector{Vector{Int64}}
  m::Int64
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  @_13::Int64
  i::Int64
  @_15::Union{Nothing, Tuple{Int64, Int64}}
  h::Int64
  line1::Matrix{Float64}
  @_18::Any
  k::Int64
  β::Any
  α::Any
  out::Any
  line2

::Matrix{Float64}
  line::Vector{Float64}
  @_25::Any
  @_26::Any
Body::Vector{Any}
1 ──        Core.NewvarNode(:(@_5))
│           Core.NewvarNode(:(@_6))
│           Core.NewvarNode(:(fragparams))
│           Core.NewvarNode(:(params))
│           (m = Main.length(Sigma))
│           (sigma = Main.map(Main.sort, Sigma))
│           (reducedsigma = sigma)
│    %8   = (1:m)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│    %9   = Base.

IteratorSize(%8)::Core.Const(Base.HasShape{1}())
│    %10  = (%9 isa Base.SizeUnknown)::Core.Const(false)
│    %11  = Core.apply_type(Main.Array, Main.Float64, 1)::Core.Const(Vector{Float64})
│    %12  = Base._array_for(%11, %8, %9)::Vector{Vector{Float64}}
│    %13  = 

Base.LinearIndices(%12)::LinearIndices{1, Tuple{Base.OneTo{Int64}}}
│           (@_13 = Base.first(%13))
│           (@_12 = Base.iterate(%8))
│    %16  = (@_12 === nothing)::Bool
│    %17  = Base.not_int(%16)::Bool
└───        goto #6 if not %17
2 ┄─ %19  = @_12::Tuple{Int64, Int64}
│           (i = Core.getfield(%19, 1))
│    %21  = Core.getfield(%19, 2)::Int64
│    %22  = Base.vect()::Vector{Any}
│           $(Expr(:inbounds, 

true))
└───        goto #4 if not %10
3 ──        Core.Const(:(Base.push!(%12, %22)))
└───        Core.Const(:(goto %28))
4 ┄─        Base.setindex!(%12, %22, @_13)
│           $(Expr(:inbounds, :pop))
│           (@_13 = Base.add_int(@_13, 1))
│           (@_12 = Base.iterate(%8, %21))
│    %31  = (@_12 === nothing)::Bool
│    %32  = Base.not_int(%31)::Bool
└───        goto #6 if not %32
5 ──        goto #2
6 ┄─        

...

60  = @_15::Tuple{Int64, Int64}
│           (k = Core.getfield(%60, 1))
│    %62  = Core.getfield(%60, 2)::Int64
│    %63  = Base.getindex(EV, k)::Vector{Int64}
│           (line2 = Base.getindex(V, Main.:(:), %63))
│    %65  = Base.getproperty(Main.Lar, :intersection)::Any
│    %66  = line1::Matrix{Float64}
│           (out = (%65)(%66, line2))
│    %68  = (out ≠ Main.nothing)::Any
└───        goto #19 if not %68
10 ─ %70  = Base.indexed_iterate(out, 1)::Any
│           (α = Core.getfield

(%70, 1))
│           (@_18 = Core.getfield(%70, 2))
│    %73  = Base.indexed_iterate(out, 2, @_18)::Any
│           (β = Core.getfield(%73, 1))
│    %75  = (0 <= α)::Any
└───        goto #12 if not %75
11 ─        (@_25 = α <= 1)
└───        goto #13
12 ─        (@_25 = false)
13 ┄        goto #19 if not @_25
14 ─ %81  = (0 <= β)::Any
└───        goto #16 if not %81
15 ─        (@_26 = β <= 1)
└───        goto #17
16 ─ 

       (@_26 = false)
17 ┄        goto #19 if not @_26
18 ─ %87  = Base.getindex(params, h)::Vector{Float64}
│           Main.push!(%87, α)
│    %89  = Base.getindex(params, k)::Vector{Float64}
└───        Main.push!(%89, β)
19 ┄        (@_15 = Base.iterate(%52, %62))
│    %92  = (@_15 === nothing)::Bool
│    %93  = Base.not_int(%92)::Bool
└───        goto #21 if not %93
20 ─        goto #9
21 ┄        (@_6 = Base.iterate(%36, %45))
│    %97  

= (@_6 === nothing)::Bool
│    %98  = Base.not_int(%97)::Bool
└───        goto #23 if not %98
22 ─        goto #7
23 ┄        (fragparams = Base.vect())
│    %102 = params::Vector{Vector{Float64}}
│           (@_5 = Base.iterate(%102))
│    %104 = (@_5 === nothing)::Bool
│    %105 = Base.not_int(%104)::Bool
└───        goto #26 if not %105
24 ┄ %107 = @_5::Tuple{Vector{Float64}, Int64}
│           (line = Core.getfield(%107, 1))
│    %109 = Core.getfield(%107, 2)::Int64
│           Main.push!(line, 0.0, 1.0)
│    %111 = Main.Set(line)::Set{Float64}
│    %112 = Main.collect(%111)::Vector{Float64}
│           (line = Main.sort(%112))
│           Main.push!(fragparams, line)
│           (@_5 = Base.iterate(%102, %109))
│    

%116 = (@_5 === nothing)::Bool
│    %117 = Base.not_int(%116)::Bool
└───        goto #26 if not %117
25 ─        goto #24
26 ┄        return fragparams



In [71]:
@benchmark linefragments(V,EV,Sigma)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  25.000 μs …  13.117 ms  ┊ GC (min … max): 0.00% … 99.38%
 Time  (median):     32.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   44.103 μs ± 181.359 μs  ┊ GC (mean ± σ):  7.70% ±  1.98%

  ▂▅▇▇█▇▅▄▄▃▃▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▂▃▃▂▂▁▂▂▂▂▂▁▁▁                     ▂
  ██████████████████████████████████████████▆▇▇▇▇▆▆▆▅▆▅▄▅▅▅▅▅▅ █
  25 μs         Histogram: log(frequency) by time       107 μs <

 Memory estimate: 25.34 KiB, allocs estimate: 800.

### Versione parallelizzata linefragments

In [72]:
function linefragments2(V,EV,sigma)
    m = length(sigma) 
    sigma = map(sort,sigma) 
    params = Array{Array{Float64,1}}(undef,m)
    for i=1:m
        params[i] = []
    end
    line1=[0.0 0.0; 0.0 0.0]
    line2=[0.0 0.0; 0.0 0.0]
    @threads for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            @threads for k in sigma[h]
            line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    if 0<=out[1]<=1 && 0<=out[2]<=1
                        push!(params[h], out[1])
                        push!(params[k], out[2])
                    end
                end
            end
        end
        end
    len = length(params)
    @threads for i=1:len
        push!(params[i], 0.0, 1.0)
        params[i] = sort(collect(Set(params[i])))
    end
    return params
end

linefragments2 (generic function with 1 method)

In [73]:
@btime linefragments2(V,EV,Sigma)

  40.400 μs (816 allocations: 26.62 KiB)


5-element Vector{Vector{Float64}}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

In [74]:
@benchmark linefragments2(V,EV,Sigma)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   40.400 μs …  14.721 ms  ┊ GC (min … max): 0.00% … 98.16%
 Time  (median):     101.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):    97.376 μs ± 273.123 μs  ┊ GC (mean ± σ):  6.01% ±  2.21%

  ▅█▆▄▃▃▁▁▃▁▁▂▁▁  █▇▅▅▅▅▃▂▁▁▁               ▁                   ▂
  ███████████████████████████████▇▇███▇▆▇▆█████▇▇▆▆▆▅▂▇▅▅▅▆▅▄▆▅ █
  40.4 μs       Histogram: log(frequency) by time        261 μs <

 Memory estimate: 26.62 KiB, allocs estimate: 816.

### Versione iniziale fragmentlines
 Funzione utilizzata per l’intersezione di coppie di segmenti di linea. In particolare, crea un indice spaziale “Sigma”, calcola i parametri d’intersezione degli spigoli e restituisce i nuovi punti generati dall’intersezione tra spigoli, tramite i parametri d’intersezione, all’interno di un array. Per N punti d’intersezione trovati, vengono generati N-1 spigoli.


In [75]:
function fragmentlines(model)
	V,EV = model
	# acceleration via spatial index computation
	Sigma = Lar.spaceindex(model)
	# actual parametric intersection of each line with the close ones
	lineparams = Lar.linefragments(V,EV,Sigma)
	# initialization of local data structures
	vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
	pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
	vertdict = OrderedDict{Array{Float64,1},Int}()
	W = Array[]
	EW = Array[]
	k = 0
	# generation of intersection points
	for (params,linepoints) in pairs
		v1 = linepoints[:,1]
		v2 = linepoints[:,2]
		points = [ v1 + t*(v2 - v1) for t in params]  
		vs = zeros(Int64,1,length(points))
		PRECISION = 8
		# identification via dictionary of points
		for (h,point) in enumerate(points)
			point = map(approxVal(PRECISION), point)
			if haskey(vertdict, point) == false
				k += 1
				vertdict[point] = k
				push!(W, point)
			end
			vs[h] = vertdict[point]
		end
		[push!(EW, [vs[k], vs[k+1]]) for k=1:length(vs)-1]
	end
	# normalization of output
	W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
	V,EV = Lar.congruence((W,EW))
	return V,EV
end

fragmentlines (generic function with 1 method)

In [76]:
@btime fragmentlines((V,EV))

  191.200 μs (2322 allocations: 97.64 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [1, 3], [3, 6]])

In [77]:
@code_warntype fragmentlines((V,EV))

MethodInstance for fragmentlines(::Tuple{Matrix{Float64}, Vector{Vector{Int64}}})
  from fragmentlines(model) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi.ipynb:1
Arguments
  #self#::Core.Const(fragmentlines)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  @_3::Any
  @_4::Any
  #83::var"#83#86"
  @_6::Int64
  k::Int64
  EW@_8::Core.Box
  W::Any
  pairs::Any
  vertdict::Union{OrderedDict{Vector{Float64}, Vector{Int64}}, OrderedDict{Vector{Float64}, Int64}}
  lineparams::Any
  Sigma::Any
  EV::Any
  V@_15::Core.Box
  #85::var"#85#88"{Matrix{Int64}}
  @_17::Union{Nothing, Tuple{Tuple{Int64, Any}, Tuple{Int64, Any}}}
  #84::var"#84#87"
  @_19::Any
  linepoints::Any
  params::Any
  PRECISION::Int64
  vs::Matrix{Int64}
  points::Any
  v2::Any
  v1::Any
  @_27::Int64
  point::Any
  h::Int64
  V@_30::Union{}
  EW@_31::Union{}
  EW@_32::Union{}
  V@_33::Union{}
Body::Tuple{Any, Any}
1 ──        Core.NewvarNode(:(@_3))


│           Core.NewvarNode(:(@_4))
│           Core.NewvarNode(:(#83))
│           Core.NewvarNode(:(k))
│           (EW@_8 = Core.Box())
│           Core.NewvarNode(:(W))
│           Core.NewvarNode(:(pairs))
│           Core.NewvarNode(:(vertdict))
│           Core.NewvarNode(:(lineparams))
│           (V@_15 = Core.Box())
│    %11  = Base.indexed_iterate(model, 1)::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(2)])
│    %12  = Core.getfield(%11, 1)::Matrix{Float64}
│    

       Core.setfield!(V@_15, :contents, %12)
│           (@_6 = Core.getfield(%11, 2))
│    %15  = Base.indexed_iterate(model, 2, @_6::Core.Const(2))::Core.PartialStruct(Tuple{Vector{Vector{Int64}}, Int64}, Any[Vector{Vector{Int64}}, Core.Const(3)])
│           (EV = Core.getfield(%15, 1))
│    %17  = Base.getproperty(Main.Lar, :spaceindex)::Any
│           (Sigma = (%17)(model))
│    %19  = Base.getproperty(Main.Lar, :linefragments)::Any
│    %20  = Core.isdefined(V@_15, :contents)::Bool


└───        goto #3 if not %20
2 ──        goto #4
3 ──        Core.NewvarNode(:(V@_30))
└───        V@_30
4 ┄─ %25  = Core.getfield(V@_15, :contents)::Any
│    %26  = EV::Vector{Vector{Int64}}
│           (lineparams = (%19)(%25, %26, Sigma))
│    %28  = Core.apply_type(

Main.Array, Main.Float64, 1)::Core.Const(Vector{Float64})
│    %29  = Core.apply_type(Main.Array, Main.Int, 1)::Core.Const(Vector{Int64})
│    %30  = Core.apply_type(Main.OrderedDict, %28, %29)::Core.Const(OrderedDict{Vector{Float64}, Vector{Int64}})
│           (vertdict = (%30)())
│    %32  = lineparams::Any
│           (#83 = %new(Main.:(var"#83#86"), V@_15))
│    %34  = #83::var"#83#86"
│    %35  = Base.Generator(%34, EV

...

│           Base.setindex!(vs, %95, h)
│           (@_17 = Base.iterate(%74, %86))
│    %98  = (@_17 === nothing)::Bool
│    %99  = Base.not_int(%98)::Bool
└───        goto #10 if not %99
9 ──        goto #6
10 ┄ %102 = Main.:(var"#85#88")::Core.Const(var"#85#88")
│    %103 = Core.typeof(vs)::Core.Const(Matrix{Int64})
│    %104 = Core.apply_type(%102, %103)::Core.Const(var"#85#88"{Matrix{Int64}})
│    %105 = EW@_8::Core.Box
│           (#85 = %new(%104, %105, vs))
│    %107 = #85::var"#85#88"{Matrix{Int64}}
│    %108 = Main

.length(vs)::Int64
│    %109 = (%108 - 1)::Int64
│    %110 = (1:%109)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│    %111 = Base.Generator(%107, %110)::Core.PartialStruct(Base.Generator{UnitRange{Int64}, var"#85#88"{Matrix{Int64}}}, Any[var"#85#88"{Matrix{Int64}}, Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])])
│           Base.collect(%111)
│           (@_4 = Base.iterate(%46, %59))
│    %114 = (@_4 === nothing)::Bool
│    %115 = Base.not_int(%114)::Bool
└───        goto #12 if not %115
11 ─        goto #5
12 ┄ %118

 = Core._apply_iterate(Base.iterate, Main.hcat, W::Vector{Array})::Any
│    %119 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Const(Vector{Int64})
│    %120 = Core.apply_type(Main.Array, %119, 1)::Core.Const(Vector{Vector{Int64}})
│    %121 = Core.isdefined(EW@_8, :contents)::Bool
└───        goto #14 if not %121
13 ─        goto #15
14 ─        Core.NewvarNode(:(EW@_31))
└───        EW@_31
15 ┄ %126 = Core.getfield(EW@_8, :contents)::Any
│    %127 = Main.convert(%120, %126)::Any
│           (W = %118)
│           Core.setfield!(EW@_8, :contents, %127)
│    %130 = Base.getproperty(Main.Lar, :congruence)::Any
│   

 %131 = W::Any
│    %132 = Core.isdefined(EW@_8, :contents)::Bool
└───        goto #17 if not %132
16 ─        goto #18
17 ─        Core.NewvarNode(:(EW@_32))
└───        EW@_32
18 ┄ %137 = Core.getfield(EW@_8, :contents)::Any
│    %138 = Core.tuple(%131, %137)::Tuple{Any, Any}
│    %139 = (%130)(%138)::Any
│    %140 = Base.indexed_iterate(%139, 1)::Any
│    %141 = Core.getfield(%140, 1)::Any
│   

        Core.setfield!(V@_15, :contents, %141)
│           (@_3 = Core.getfield(%140, 2))
│    %144 = Base.indexed_iterate(%139, 2, @_3)::Any
│           (EV = Core.getfield(%144, 1))
│    %146 = Core.isdefined(V@_15, :contents)::Bool
└───        goto #20 if not %146
19 ─        goto #21
20 ─        Core.NewvarNode(:(V@_33))
└───        V@_33
21 ┄ %151 = Core.getfield(V@_15, :contents)::Any
│    %152 = Core.tuple(%151, EV)::Tuple{Any, Any}
└───        return %152



In [78]:
@benchmark fragmentlines((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  191.200 μs …  15.786 ms  ┊ GC (min … max): 0.00% … 94.92%
 Time  (median):     243.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   358.736 μs ± 531.643 μs  ┊ GC (mean ± σ):  5.87% ±  4.07%

  ▂▆█▇█▇▅▄▂▂▁▂▁   ▁              ▃▇▇▅▄▃▃▂▂▂▁                    ▂
  █████████████████████▇▇█▆▇▆▆▆▆▅██████████████▇▇▇▇▆▆▆▆▅▅▅▅▅▄▃▅ █
  191 μs        Histogram: log(frequency) by time        741 μs <

 Memory estimate: 97.64 KiB, allocs estimate: 2322.

### Versione parallelizzata fragmentlines

In [79]:
function fragmentlines2(model)
    V,EV = model
    Sigma = spaceindex(model)
    lineparams = linefragments(V,EV,Sigma)
    vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
    pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
    vertdict = OrderedDict{Array{Float64,1},Int}()
    W = Array[]
    EW = Array[]
    k = 0
    l = length(pairs)
    @inbounds for i = 1:l
        params = pairs[i][1]
        linepoints = pairs[i][2]
        v1 = linepoints[:,1]    # Isolo primo punto dello spigolo
        v2 = linepoints[:,2]    # Isolo secondo punto dello spigolo
        points = [ v1 + t*(v2 - v1) for t in params] 
        vs = zeros(Int64,1,length(points))
        PRECISION = 8
        numpoint = length(points)
        @inbounds for h = 1:numpoint
            points[h] = map(approxVal(PRECISION), points[h])
            if !haskey(vertdict, points[h])
                k += 1  # Genero ID punto 
                vertdict[points[h]] = k     # Associo l'ID al punto
                push!(W, points[h])         # Effettua una push del punto(x,y) nell'array W
            end
            vs[h] = vertdict[points[h]] 
        end
        m = length(vs) - 1
        @inbounds for k=1:m
            push!(EW, [vs[k], vs[k+1]])
        end
    end
    W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
    V,EV = congruence((W,EW))
    return V,EV
end


fragmentlines2 (generic function with 1 method)

In [80]:
@btime fragmentlines2((V,EV))

  183.800 μs (2270 allocations: 95.69 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [1, 3], [3, 6]])

In [81]:
@benchmark fragmentlines2((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  182.800 μs …  16.004 ms  ┊ GC (min … max): 0.00% … 97.78%
 Time  (median):     244.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   357.882 μs ± 541.196 μs  ┊ GC (mean ± σ):  6.03% ±  4.08%

   ▄▆▆▇█▇▅▃▂▂▂▁▁▁▁ ▁              ▃█▇▅▄▃▃▂▁▂▁▁                  ▂
  ███████████████████▇█▇▆▇█▇▆▆▆▇▆▆████████████▇▇██▇█▇▇▆▆▇▇▅▆▆▅▅ █
  183 μs        Histogram: log(frequency) by time        695 μs <

 Memory estimate: 95.69 KiB, allocs estimate: 2270.

### Funzione fraglines

In [82]:
function fraglines(sx::Float64=1.2,sy::Float64=1.2,sz::Float64=1.2)
	function fraglines0(model)
		V,EV = Lar.fragmentlines(model)

		W = zeros(Float64, size(V,1), 2*length(EV))
		EW = Array{Array{Int64,1},1}()
		for (k,(v1,v2)) in enumerate(EV)
			if size(V,1)==2
				x,y = (V[:,v1] + V[:,v2]) ./ 2
				scx,scy = x*sx, y*sy
				t = [scx-x, scy-y]
			elseif size(V,1)==3
				x,y,z = (V[:,v1] + V[:,v2]) ./ 2
				scx,scy,scz = x*sx, y*sy, z*sz
				t = [scx-x, scy-y, scz-z]
			end
			W[:,2*k-1] = V[:,v1] + t
			W[:,2*k] = V[:,v2] + t
			push!(EW, [2*k-1, 2*k])
		end
		return W,EW
	end
	return fraglines0
end


fraglines (generic function with 4 methods)

## Analisi funzioni aggiuntive create

### createIntervalTree

In [83]:
@btime createIntervalTree(dict)

  910.000 ns (16 allocations: 3.00 KiB)


IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

In [84]:
@benchmark createIntervalTree(dict)

BenchmarkTools.Trial: 10000 samples with 33 evaluations.
 Range (min … max):  872.727 ns … 201.709 μs  ┊ GC (min … max):  0.00% … 98.77%
 Time  (median):       1.053 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.433 μs ±   4.071 μs  ┊ GC (mean ± σ):  10.87% ±  4.14%

  ▂▇██▇▅▄▃▃▃▃▃▃▃▂▂▃▃▃▂▁▂▁▁▁ ▂▂▁   ▁  ▁                          ▂
  ██████████████████████████████████████▇▇▇▆▆▆▇▇▇▆▆▅▆▆▅▆▄▆▁▄▃▅▆ █
  873 ns        Histogram: log(frequency) by time       3.84 μs <

 Memory estimate: 3.00 KiB, allocs estimate: 16.

### removeIntersection

In [85]:
cov = boxcovering2(bb, 1, t)

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [86]:
@btime removeIntersection(cov)

  3.438 μs (46 allocations: 3.52 KiB)


In [87]:
@benchmark removeIntersection(cov)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.625 μs …  1.171 ms  ┊ GC (min … max): 0.00% … 99.35%
 Time  (median):     4.425 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.314 μs ± 23.538 μs  ┊ GC (mean ± σ):  9.84% ±  2.22%

   ▁▄▅▆▇█▇▇▅▄▃▃▂▁▂▂▁▁▁▁▁▂▁▂▁▁   ▁▁▁▁ ▁                       ▂
  ▅███████████████████████████████████▇▇▇▇▅▅▆▆▇▅▆▆▆▅▅▆▅▆▅▄▅▅ █
  3.62 μs      Histogram: log(frequency) by time       10 μs <

 Memory estimate: 3.52 KiB, allocs estimate: 46.

### addIntersection

In [88]:
@btime addIntersection(cov, 1, it)

  1.860 μs (12 allocations: 1.05 KiB)


In [89]:
@benchmark addIntersection(cov, 1, it)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.890 μs …  1.112 ms  ┊ GC (min … max): 0.00% … 99.54%
 Time  (median):     2.410 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.812 μs ± 11.138 μs  ┊ GC (mean ± σ):  3.94% ±  1.00%

  ▁▄▆▇██▇▇▆▆▅▄▃▃▂▂▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁                           ▃
  █████████████████████████████████████▇▇▇▇▆▆▇▇▇▆▆▆▆▆▆▇▅▆▃▅▄ █
  1.89 μs      Histogram: log(frequency) by time     6.53 μs <

 Memory estimate: 1.05 KiB, allocs estimate: 12.